In [4]:
import time
import numpy as np
import pygame
from NovelSwarmBehavior.novel_swarms.config.AgentConfig import DiffDriveAgentConfig
from NovelSwarmBehavior.novel_swarms.novelty.BehaviorDiscovery import BehaviorDiscovery
from NovelSwarmBehavior.novel_swarms.config.EvolutionaryConfig import GeneticEvolutionConfig
from NovelSwarmBehavior.novel_swarms.config.WorldConfig import RectangularWorldConfig
from NovelSwarmBehavior.novel_swarms.config.defaults import ConfigurationDefaults
from NovelSwarmBehavior.novel_swarms.novelty.GeneRule import GeneRule
from NovelSwarmBehavior.novel_swarms.config.OutputTensorConfig import OutputTensorConfig
from NovelSwarmBehavior.novel_swarms.sensors.BinaryLOSSensor import BinaryLOSSensor
from NovelSwarmBehavior.novel_swarms.sensors.SensorSet import SensorSet
from data.swarmset import SwarmDataset, DataBuilder
from generation.halted_evolution import HaltedEvolution

def build_random_dataset():
    sensors = SensorSet([
        BinaryLOSSensor(angle=-(np.pi/4)),
        # BinaryFOVSensor(theta=14 / 2, distance=125, degrees=True)
    ])

    agent_config = DiffDriveAgentConfig(
        sensors=sensors,
    )

    genotype = [
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
    ]

    phenotype = ConfigurationDefaults.BEHAVIOR_VECTOR

    world_config = RectangularWorldConfig(
        size=(500, 500),
        n_agents=24,
        behavior=phenotype,
        agentConfig=agent_config,
        padding=15
    )

    novelty_config = GeneticEvolutionConfig(
        gene_rules=genotype,
        phenotype_config=phenotype,
        n_generations=100,
        n_population=100,
        crossover_rate=0.7,
        mutation_rate=0.15,
        world_config=world_config,
        k_nn=8,
        simulation_lifespan=800,
        display_novelty=False,
        save_archive=False,
        show_gui=True
    )

    pygame.init()
    pygame.display.set_caption("Evolutionary Novelty Search")
    screen = pygame.display.set_mode((world_config.w, world_config.h))

    output_config = OutputTensorConfig(
        timeless=True,
        total_frames=80,
        steps_between_frames=2,
        screen=screen
    )

    halted_evolution = HaltedEvolution(
        world=world_config,
        evolution_config=novelty_config,
        output_config=output_config
    )

    baseline_data = DataBuilder("data/full-fixed-theta", steps=1200, agents=24, ev=halted_evolution, screen=screen)
    baseline_data.create()
    baseline_data.evolution.close()

build_random_dataset()

Size of Genome scope: 100000000
0.0% Complete
0.02% Complete
0.04% Complete
0.06% Complete
0.08% Complete
0.1% Complete
0.12% Complete
0.14% Complete
0.16% Complete
0.18% Complete
0.2% Complete
0.22% Complete
0.24% Complete
0.26% Complete
0.28% Complete
0.3% Complete
0.32% Complete
0.34% Complete
0.36% Complete
0.38% Complete
0.4% Complete
0.42% Complete
0.44% Complete
0.46% Complete
0.48% Complete
0.5% Complete
0.52% Complete
0.54% Complete
0.56% Complete
0.58% Complete
0.6% Complete
0.62% Complete
0.64% Complete
0.66% Complete
0.68% Complete
0.7% Complete
0.72% Complete
0.74% Complete
0.76% Complete
0.78% Complete
0.8% Complete
0.82% Complete
0.84% Complete
0.86% Complete
0.88% Complete
0.9% Complete
0.92% Complete
0.94% Complete
0.96% Complete
0.98% Complete
1.0% Complete
1.02% Complete
1.04% Complete
1.06% Complete
1.08% Complete
1.1% Complete
1.12% Complete
1.14% Complete
1.16% Complete
1.18% Complete
1.2% Complete
1.22% Complete
1.24% Complete
1.26% Complete
1.28% Complete
1.3% C

# Embed into Latent Space

In [1]:
import torch
from data.swarmset import ContinuingDataset, SwarmDataset
from networks.embedding import NoveltyEmbedding
from generation.evolution import ModifiedHaltingEvolution
from networks.archive import DataAggregationArchive
from hil.HIL import HIL

def update_network(network, loss_fn, optim, anchor_image, pos_image, neg_image):
    optim.zero_grad()
    anchor_out, pos_out, neg_out = network.network_with_transforms(anchor_image, pos_image, neg_image)
    loss = loss_fn(anchor_out, pos_out, neg_out)
    loss.backward()
    optim.step()
    return loss

def pretraining(data, network, loss_fn, optim, data_size=500):
    samples = np.random.random_integers(0, len(data) - 1, (data_size, 2))
    total_loss = 0
    total_updates = 0
    for i, s in enumerate(samples):
        if i % 50 == 0:
            print(f"Unsupervised Training.. {(i * 100) / data_size}")
        anchor_image = data[s[0]][0]
        pos_image = data[s[0]][0]


        if len(data) - 2 > s[1] > 2:
            neg_images = [
                # data[s[1] - 2][0],
                data[s[1] - 1][0],
                data[s[1]][0],
                # data[s[1] + 1][0],
                # data[s[1] + 2][0],
            ]
        else:
            neg_images = [
                data[s[1]][0]
            ]

        for neg in neg_images:
            loss = update_network(network, loss_fn, optim, anchor_image, pos_image, neg)
            total_loss += loss.item()
            total_updates += 1

    return total_loss / total_updates


def human_in_the_loop(anchor_dataset, network, optimizer, loss_fn, HIL_archive, random_archive, stop_at):
    print("HIL TIME!")
    improvements, human_loss, triplet_helpfulness, embedded_archive = hil.humanInput(anchor_dataset, network, optimizer, loss_fn, HIL_archive, random_archive, stop_at)
    print(f"Improvement Count: {improvements}, loss: {human_loss}")

    HIL_archive.save_to_file(f"data/queries/{trial_name}_hil.csv")
    random_archive.save_to_file(f"data/queries/{trial_name}_rand.csv")
    return improvements, human_loss, triplet_helpfulness, embedded_archive

pygame 2.1.2 (SDL 2.0.16, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import torch
from data.swarmset import ContinuingDataset, SwarmDataset
from networks.embedding import NoveltyEmbedding
from generation.evolution import ModifiedHaltingEvolution
from networks.archive import DataAggregationArchive
from hil.HIL import HIL

import numpy as np
import time
from scipy import ndimage
from torch.utils.tensorboard import SummaryWriter

trial_name = f"{str(int(time.time()))}"

TRAIN = True
CLUSTER_AND_DISPLAY = True
WRITE_OUT = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

writer = SummaryWriter()
network = NoveltyEmbedding().to(device)

SAVE_CLUSTER_IMAGES = True
SAVE_CLUSTER_MEDOIDS = True
PRETRAINING = True
HUMAN_IN_LOOP = True
SYNTHETIC_HIL = True
EVOLUTION = True
EPOCHS = 20
DATA_SIZE = 10000
EVOLUTIONS_PER_EPOCH = 12
K_CLUSTERS = 8

anchor_dataset = ContinuingDataset("data")
sampled_dataset = SwarmDataset("data/full", rank=0)
# evolution, _ = ModifiedHaltingEvolution.defaultEvolver(steps=800, n_agents=24, evolve_population=100, seed=None)
evolution, _ = ModifiedHaltingEvolution.defaultEvolver(steps=1200, n_agents=24, evolve_population=80, seed=None)
optimizer = torch.optim.Adam(network.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.8)

# Margin was 10
loss_fn = torch.nn.TripletMarginLoss(margin=15)
hil = HIL(name=trial_name, synthetic=SYNTHETIC_HIL, data_limiter=DATA_SIZE, clusters=K_CLUSTERS)
HIL_archive = DataAggregationArchive()
random_archive = DataAggregationArchive(scalar=True)
EPSILON = 0.5

simulation_time = 0
evolution_time = 0
training_time = 0
hil_time = 0

# TODO: Add Randomly-sampled Contrastive/Triplet Loss
if PRETRAINING:
    network.load_model("unsupervised_decay_0.7_target_0.08")
    print("Pretrained model loaded!")

dataset = anchor_dataset if EVOLUTION else sampled_dataset

if TRAIN:
    STOP_FLAG = False
    for epoch in range(EPOCHS):
        if STOP_FLAG:
            break

        start_time = time.time()
        if EVOLUTION:
            for gen in range(EVOLUTIONS_PER_EPOCH if epoch > 0 else 3):
                # Simulate current population + Save Data
                for i in range(len(evolution.getPopulation())):
                    # The collection of the original behavior vector below is only used to collect data to compare with the baseline
                    visual_behavior, genome, baseline_behavior = evolution.next()
                    dataset.new_entry(visual_behavior, genome, baseline_behavior)
                simulation_time += (time.time() - start_time)

                # Then, evolve
                start_time = time.time()
                embedded_archive = hil.getEmbeddedArchive(dataset, network)
                evolution.overwriteArchive(embedded_archive, random_archive)
                embedded_behavior = embedded_archive.archive[-evolution.evolve_config.population:]
                evolution.overwriteBehavior(embedded_behavior)
                evolution.evolve()
                evolution.restart_screen()
                evolution_time += (time.time() - start_time)

                print(f"Evolution complete for e{epoch} and gen{gen}")

                # Record the accuracy of the medoids with respect to the synthetic policy
                medoid_acc, cluster_acc = 0, 0
                if len(dataset) > 0 and SAVE_CLUSTER_MEDOIDS:
                    if EVOLUTION:
                        hil.synthetic_knowledge = hil.syntheticBehaviorSpace(dataset)
                        print(f"Synthetic Human Knowledge Size: {len(hil.synthetic_knowledge.labels_)}")
                    medoid_acc, cluster_acc = hil.record_medoids(network, dataset)

                # Cluster current dataset, display clusters, and save for analysis
                if len(dataset) > 0 and SAVE_CLUSTER_IMAGES:
                    hil.embed_and_cluster(network, dataset, auto_quit=True)
                    evolution.restart_screen()

        start_time = time.time()
        # Human in the Loop determines behavior embedding
        if HUMAN_IN_LOOP:
            improvements, human_loss, triplet_helpfulness, _ =   human_in_the_loop(
                                                                dataset,
                                                                network,
                                                                optimizer,
                                                                loss_fn,
                                                                HIL_archive,
                                                                random_archive,
                                                                len(dataset)
                                                            )
            hil_time += (time.time() - start_time)

        # Train on past user information
        start_time = time.time()
        loss = None

        average_loss = 50
        loop = 0
        TRANSFORMS = 20
        network.train()
        while (average_loss > 0.01) and loop < 20:

            loss_sum = 0
            total_loss = 0

            for i, (anchor, pos, neg) in enumerate(HIL_archive):
                anchor_encoding = dataset[anchor][0]
                similar_encoding = dataset[pos][0]
                anti_encoding = dataset[neg][0]
                optimizer.zero_grad()

                anchor_out, pos_out, neg_out = network.network_from_numpy(anchor_encoding, similar_encoding, anti_encoding)
                loss = loss_fn(anchor_out, pos_out, neg_out)
                loss_sum += loss.item()
                if loss.item() > 0:
                    loss.backward()
                    optimizer.step()
                total_loss += 1

                # anchor_out, pos_out, neg_out = network.network_with_transforms(anchor_encoding, similar_encoding, anti_encoding)
                # loss = loss_fn(anchor_out, pos_out, neg_out)
                # loss_sum += loss.item()
                # if loss.item() > 0:
                #     loss.backward()
                #     optimizer.step()
                # total_loss += 1

                if loss is not None and i > 0 and i % 20 == 0:
                    print(f"Epoch Progress: {(i*100) / len(HIL_archive)}%, Immediate Loss: {loss.item()}")

            average_loss = loss_sum / (total_loss + 1)
            print(f"Loop: {loop}, Average Loss: {average_loss}")
            loop += 1

        print(f"Epoch: {epoch}")
        training_time += (time.time() - start_time)

        writer.add_scalar("Loss/Average", loss_sum / (total_loss + 1), epoch)

        if SAVE_CLUSTER_MEDOIDS:
            writer.add_scalar("Accuracy/MedoidClassification", medoid_acc, epoch)
            writer.add_scalar("Accuracy/RandomSampleClassification", cluster_acc, epoch)

        if HUMAN_IN_LOOP:
            writer.add_scalar("Loss/TripletQuality", triplet_helpfulness, epoch)
            writer.add_scalar("Queries/Total_Human_Queries", epoch*8*9, epoch)
            writer.add_scalar("Queries/Total_Triplets_Generated", len(HIL_archive), epoch)
            writer.add_scalar("Queries/Total_Random_Classes", len(random_archive), epoch)

        if EVOLUTION:
            writer.add_scalar("Novelty/Highest", evolution.behavior_discovery.getBestScore(), epoch)
            writer.add_scalar("Novelty/Average", evolution.behavior_discovery.getAverageScore(), epoch)

        writer.add_scalar("Time/Simulation", simulation_time, epoch)
        writer.add_scalar("Time/HIL", hil_time, epoch)
        writer.add_scalar("Time/Evolution", evolution_time, epoch)
        writer.add_scalar("Time/Training", training_time, epoch)

        scheduler.step()

evolution.close()

HIL Init!
Pretrained model loaded!
BLENDING: 63.21501406699842
BLENDING: 60.929972669733964
BLENDING: 42.82481163467072
BLENDING: 39.95620451725868
BLENDING: 39.071279337262894
BLENDING: 42.35692541866617
BLENDING: 40.65277959717089
BLENDING: 69.21484604308567
BLENDING: 31.82506158050128
BLENDING: 48.63639881478059
BLENDING: 62.959465358067504
BLENDING: 39.81248321765212
BLENDING: 35.55443901110121
BLENDING: 83.0456750646794
BLENDING: 29.326388317769158
BLENDING: 72.66471765841801
BLENDING: 49.74667556504483
BLENDING: 34.19702001295139
BLENDING: 72.49159209738247
BLENDING: 33.22672935810525
BLENDING: 50.6236591458693
BLENDING: 74.17094094239032
BLENDING: 48.191975105347716
BLENDING: 38.94371221859983
BLENDING: 31.47520439727015
BLENDING: 33.629153020051994
BLENDING: 37.58333047090114
BLENDING: 85.99131353474496
BLENDING: 74.16476767627287
BLENDING: 63.800608293803116
BLENDING: 38.74202102149504
BLENDING: 57.94774835976383
BLENDING: 50.37744139396904
BLENDING: 39.386218137982496
BLENDIN

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


BLENDING: 42.0301946248511
BLENDING: 35.179760161328275
BLENDING: 47.603355871360684
BLENDING: 30.525216508914674
BLENDING: 34.83543219733086
BLENDING: 41.319065047949465
BLENDING: 123.91156278677812
BLENDING: 44.27037998289349
BLENDING: 27.51673322066846
BLENDING: 40.20746831423697
BLENDING: 39.14211884993512
BLENDING: 30.390101642809057
BLENDING: 33.77864634742983
BLENDING: 27.214206302491398
BLENDING: 28.75133032522045
BLENDING: 24.370038058530774
BLENDING: 47.58036330528683
BLENDING: 51.29941371944195
BLENDING: 28.341939298737046
BLENDING: 23.956967280723106
BLENDING: 26.189546896073654
BLENDING: 43.99961493632246
BLENDING: 33.57478150451705
BLENDING: 46.169430030253494
BLENDING: 37.38487951026502
BLENDING: 54.788401123403204
BLENDING: 77.01321459220978
BLENDING: 30.212823271694173
BLENDING: 49.549194365631166
BLENDING: 66.9723165915777
BLENDING: 29.52292016356073
BLENDING: 22.95836908750328
BLENDING: 70.25909367615346
BLENDING: 28.133546512408383
BLENDING: 33.515976412785655
BLEND

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


BLENDING: 39.10135431841036
BLENDING: 63.49782166097922
BLENDING: 47.10461001346228
BLENDING: 42.06432287455089
BLENDING: 27.260386141318364
BLENDING: 36.29450897684426
BLENDING: 30.378365290154584
BLENDING: 33.538529009280914
BLENDING: 62.716283720625015
BLENDING: 31.593984075766045
BLENDING: 55.21731875320856
BLENDING: 69.65275126827413
BLENDING: 27.94765046590759
BLENDING: 26.149429896521376
BLENDING: 27.152088772887176
BLENDING: 47.81306508046679
BLENDING: 31.475202686479513
BLENDING: 31.885710404628085
BLENDING: 43.37078108293563
BLENDING: 48.93490036475611
BLENDING: 51.84528665020597
BLENDING: 32.16652690370526
BLENDING: 40.41024407194717
BLENDING: 26.777345541091076
BLENDING: 36.58967304350981
BLENDING: 37.19318138688433
BLENDING: 29.695414922472093
BLENDING: 30.459785497427518
BLENDING: 25.3162866354473
BLENDING: 31.989351146073705
BLENDING: 32.91810337791182
BLENDING: 54.09218977206832
BLENDING: 28.789372944372307
BLENDING: 48.139523941410786
BLENDING: 26.68935401931284
BLENDI

...

BLENDING: 1.7295403872744737
BLENDING: 2.388820976369728
BLENDING: 6.690254234261949
BLENDING: 4.392095711580931
BLENDING: 2.6591591244537534
BLENDING: 2.1789363218273285
BLENDING: 10.955522212699766
BLENDING: 2.255761265554885
BLENDING: 1.6862917815316625
BLENDING: 3.683718451877732
BLENDING: 2.6626845080624864
BLENDING: 3.8662279246682756
BLENDING: 2.964258053890526
BLENDING: 1.678108259232061
BLENDING: 3.1894488604713787
BLENDING: 8.362217528606129
BLENDING: 2.4673025063580383
BLENDING: 4.009942820866881
BLENDING: 2.9823447458151495
BLENDING: 2.793464381352982
BLENDING: 3.112449701231087
BLENDING: 2.6796724333206234
BLENDING: 3.854772906761539
BLENDING: 2.10772309874481
BLENDING: 2.409339283405237
BLENDING: 2.0852637112039547
BLENDING: 2.001533817840555
BLENDING: 4.2395664604124335
BLENDING: 2.5193191747659864
BLENDING: 2.9290078403818556
BLENDING: 2.6128490343403046
BLENDING: 2.438094876227808
BLENDING: 2.8813725206961314
BLENDING: 3.4568779623127877
BLENDING: 2.1868230609886967
BL

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


BLENDING: 2.8589153964749605
BLENDING: 2.476059314988851
BLENDING: 14.24689727269246
BLENDING: 2.425552053717956
BLENDING: 2.7106248399441926
BLENDING: 4.166865191235855
BLENDING: 3.025521297457752
BLENDING: 3.085445545029602
BLENDING: 2.1092582671224465
BLENDING: 1.650056004490009
BLENDING: 11.807547935202376
BLENDING: 1.9031442618594239
BLENDING: 1.7794136217081298
BLENDING: 3.023748288681308
BLENDING: 2.5144671873233304
BLENDING: 2.7083103231491723
BLENDING: 3.0936891712335783
BLENDING: 2.854809221298118
BLENDING: 1.8558931507816017
BLENDING: 2.5083979276929336
BLENDING: 2.9954500809821156
BLENDING: 3.2736512099177792
BLENDING: 4.092961196308881
BLENDING: 3.1830754628228264
BLENDING: 2.7526522457910234
BLENDING: 2.287401465416194
BLENDING: 5.107933465313649
BLENDING: 2.41685872848113
BLENDING: 1.9876259726048948
BLENDING: 1.6193835476726721
BLENDING: 2.640426116367027
BLENDING: 3.5306155748328663
BLENDING: 2.2653512906779962
BLENDING: 3.6124470990692044
BLENDING: 2.999041282748123
B

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


KeyboardInterrupt: 

In [3]:
import pygame
pygame.quit()

# Pretraining

In [9]:
# Save Model
target = 0.08
loss = 50
network = NoveltyEmbedding().to(device)
optimizer = torch.optim.Adam(network.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)
if PRETRAINING:
    epochs = 0
    while loss > target:
        loss = pretraining(sampled_dataset, network, loss_fn, optimizer, data_size=1000)
        print(f"Epoch {epochs}, loss: {loss}")
        epochs += 1
        scheduler.step()

network.save_model()

/tmp/ipykernel_1091999/39441780.py:17: DeprecationWarning: This function is deprecated. Please call randint(0, 9999 + 1) instead
  samples = np.random.random_integers(0, len(data) - 1, (data_size, 2))


Unsupervised Training.. 0.0
Unsupervised Training.. 5.0
Unsupervised Training.. 10.0
Unsupervised Training.. 15.0
Unsupervised Training.. 20.0
Unsupervised Training.. 25.0
Unsupervised Training.. 30.0
Unsupervised Training.. 35.0
Unsupervised Training.. 40.0
Unsupervised Training.. 45.0
Unsupervised Training.. 50.0
Unsupervised Training.. 55.0
Unsupervised Training.. 60.0
Unsupervised Training.. 65.0
Unsupervised Training.. 70.0
Unsupervised Training.. 75.0
Unsupervised Training.. 80.0
Unsupervised Training.. 85.0
Unsupervised Training.. 90.0
Unsupervised Training.. 95.0
Epoch 0, loss: 12.100919920444488
Unsupervised Training.. 0.0
Unsupervised Training.. 5.0
Unsupervised Training.. 10.0
Unsupervised Training.. 15.0
Unsupervised Training.. 20.0
Unsupervised Training.. 25.0
Unsupervised Training.. 30.0
Unsupervised Training.. 35.0
Unsupervised Training.. 40.0
Unsupervised Training.. 45.0
Unsupervised Training.. 50.0
Unsupervised Training.. 55.0
Unsupervised Training.. 60.0
Unsupervised 

KeyboardInterrupt: 

In [10]:
network.save_model()

In [ ]:
print(optimizer.lr)

In [ ]:
import time
import torch
from data.swarmset import SwarmDataset, DataBuilder
from networks.embedding import NoveltyEmbedding
from NovelSwarmBehavior.novel_swarms.novelty.NoveltyArchive import NoveltyArchive
from NovelSwarmBehavior.novel_swarms.config.ResultsConfig import ResultsConfig
from NovelSwarmBehavior.novel_swarms.results.results import main as results
from NovelSwarmBehavior.novel_swarms.config.defaults import ConfigurationDefaults
from data.swarmset import SwarmDataset, DataBuilder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network = NoveltyEmbedding().to(device)
network.load_model("trialA-10-18-2022")
# anchor_dataset = SwarmDataset("data/full", rank=0)

WRITE_OUT = True
if WRITE_OUT:
    network.eval()
    test_archive = NoveltyArchive()
    for i in range(len(anchor_dataset)):
        anchor_encoding, genome, _ = anchor_dataset[i]
        anchor_encoding = torch.from_numpy(anchor_encoding).to(device).float()
        embedding = network(anchor_encoding.unsqueeze(0)).squeeze(0).cpu().detach().numpy()
        test_archive.addToArchive(embedding, genome)

In [ ]:
from ui.clustering_gui import ClusteringGUI
import pygame

agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT
world_config = ConfigurationDefaults.RECTANGULAR_WORLD
world_config.addAgentConfig(agent_config)
config = ResultsConfig(archive=test_archive, k_clusters=6, world_config=world_config, tsne_perplexity=16, tsne_early_exaggeration=12, skip_tsne=False)
gui = ClusteringGUI(config)
gui.displayGUI()
# results(config)
pygame.quit()

# Clustering + Analysis

In [ ]:
from ui.clustering_gui import ClusteringGUI

# Cluster over saved behaviors
archive = NoveltyArchive()
for i, (_, genome, behavior, _, _, _, _) in enumerate(anchor_dataset):
    archive.addToArchive(vec=behavior, genome=genome)

agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT
world_config = ConfigurationDefaults.RECTANGULAR_WORLD
world_config.addAgentConfig(agent_config)
config = ResultsConfig(archive=archive, k_clusters=7, world_config=world_config, tsne_perplexity=20, tsne_early_exaggeration=2, skip_tsne=False)

gui = ClusteringGUI(config)
gui.displayGUI()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(human_l_hist, "b", label='Human Loss')
plt.ylabel("Loss")
plt.xlabel("Time (Epochs)")
plt.legend()